In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

from covidcaremap.constants import state_name_to_abbreviation
from covidcaremap.geo import sum_per_county, sum_per_state, sum_per_hrr
from covidcaremap.data import (external_data_path,
                               processed_data_path,
                               read_census_data_df)

# Merge Region and Census Data

This notebook utilizes US Census data at the county and state level to merge population data into the county, state, and HRR region data.

Most logic taken from [usa_beds_capacity_analysis_20200313_v2](https://github.com/daveluo/covid19-healthsystemcapacity/blob/9a45c424a23e7a15559527893ebeb28703f26422/nbs/usa_beds_capacity_analysis_20200313_v2.ipynb)

In [2]:
census_df = read_census_data_df()

#### Format FIPS code as to be joined with county geo data

In [3]:
census_df['fips_code'] = census_df['STATE'].apply(lambda x: str(x).zfill(2)) + \
                            census_df['COUNTY'].apply(lambda x: str(x).zfill(3))

#### Filter to 7/1/2018 population estimate

In [4]:
census2018_df = census_df[census_df['YEAR'] == 11]

#### Filter by age groups

We will be looking at total population, adult population (20+ years old), 
and elderly population (65+ years old). These age groups match up with the
CDC groupings here: https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm?s_cid=mm6912e2_w

From https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2018/cc-est2018-alldata.pdf, the key for AGEGRP is as follows:
- 0 = Total
- 1 = Age 0 to 4 years
- 2 = Age 5 to 9 years
- 3 = Age 10 to 14 years
- 4 = Age 15 to 19 years
- 5 = Age 20 to 24 years
- 6 = Age 25 to 29 years
- 7 = Age 30 to 34 years
- 8 = Age 35 to 39 years
- 9 = Age 40 to 44 years
- 10 = Age 45 to 49 years
- 11 = Age 50 to 54 years
- 12 = Age 55 to 59 years
- 13 = Age 60 to 64 years
- 14 = Age 65 to 69 years
- 15 = Age 70 to 74 years
- 16 = Age 75 to 79 years
- 17 = Age 80 to 84 years
- 18 = Age 85 years or older

In [5]:
county_pop_all = census2018_df[census2018_df['AGEGRP']==0].groupby(
    ['fips_code'])['TOT_POP'].sum()
county_pop_adult = census2018_df[census2018_df['AGEGRP']>=5].groupby(
    ['fips_code'])['TOT_POP'].sum()
county_pop_elderly = census2018_df[census2018_df['AGEGRP']>=14].groupby(
    ['fips_code'])['TOT_POP'].sum()

In [6]:
county_pop_all.sum(), county_pop_adult.sum(), county_pop_elderly.sum()

(327167434, 245184769, 52431193)

In [7]:
state_pop_all = census2018_df[census2018_df['AGEGRP']==0].groupby(
    ['STNAME'])['TOT_POP'].sum()
state_pop_adult = census2018_df[census2018_df['AGEGRP']>=5].groupby(
    ['STNAME'])['TOT_POP'].sum()
state_pop_elderly = census2018_df[census2018_df['AGEGRP']>=14].groupby(
    ['STNAME'])['TOT_POP'].sum()

In [8]:
state_pop_all.sum(), state_pop_adult.sum(), state_pop_elderly.sum()

(327167434, 245184769, 52431193)

In [9]:
county_pops = {
    'Population': county_pop_all,
    'Population (20+)': county_pop_adult,
    'Population (65+)': county_pop_elderly
}

state_pops = {
    'Population': state_pop_all,
    'Population (20+)': state_pop_adult,
    'Population (65+)': state_pop_elderly
}

In [10]:
def set_population_field(target_df, pop_df, column_name, join_on):
    result = target_df.join(pop_df, how='left', on=join_on)
    result = result.rename({'TOT_POP': column_name}, axis=1)
    result = result.fillna(value={column_name: 0})
    return result

### Merge census data into states

In [11]:
state_gdf = gpd.read_file(external_data_path('us_states.geojson'), encoding='utf-8')

In [12]:
enriched_state_df = state_gdf.set_index('NAME')
for column_name, pop_df in state_pops.items():
    pop_df = pop_df.rename({'STNAME': 'State Name'}, axis=1)
    enriched_state_df = set_population_field(enriched_state_df,
                                              pop_df, 
                                              column_name, 
                                              join_on='NAME')
enriched_state_df = enriched_state_df.reset_index()
enriched_state_df = enriched_state_df.rename(columns={'STATE': 'STATE_FIPS',
                                                      'NAME': 'State Name'})
enriched_state_df['State'] = enriched_state_df['State Name'].apply(
    lambda x: state_name_to_abbreviation[x])


In [13]:
enriched_state_df.to_file(processed_data_path('us_states_with_pop.geojson'), driver='GeoJSON')

### Merge census data into counties

In [14]:
county_gdf = gpd.read_file(external_data_path('us_counties.geojson'), encoding='utf-8')


In [15]:
county_gdf = county_gdf.rename(columns={'STATE': 'STATE_FIPS',
                                        'NAME': 'County Name'})

In [16]:
county_gdf = county_gdf.merge(enriched_state_df[['STATE_FIPS', 'State']], on='STATE_FIPS')

In [17]:
#  FIPS code is last 5 digits of GEO_ID
county_gdf['COUNTY_FIPS'] = county_gdf['GEO_ID'].apply(lambda x: x[-5:])
county_gdf = county_gdf.drop(columns=['COUNTY'])

In [18]:
enriched_county_df = county_gdf
for column_name, pop_df in county_pops.items():
    enriched_county_df = set_population_field(enriched_county_df,
                                              pop_df, 
                                              column_name, 
                                              join_on='COUNTY_FIPS')

In [19]:
enriched_county_df.to_file(processed_data_path('us_counties_with_pop.geojson'), driver='GeoJSON')

## Generate population data for HRRs

Spatially join HRRs with counties. For each intersecting county, take the ratio of the area of intersection with the HRR and the area of the county as the ratio of population for that county to be assigned to that HRR.

In [20]:
hrr_gdf = gpd.read_file(external_data_path('us_hrr.geojson'), encoding='utf-8')
hrr_gdf = hrr_gdf.to_crs('EPSG:5070')
hrr_gdf['hrr_geom'] = hrr_gdf['geometry']

In [21]:
county_pop_gdf = enriched_county_df
county_pop_gdf = county_pop_gdf.to_crs('EPSG:5070')
county_pop_gdf['county_geom'] = county_pop_gdf['geometry']

In [22]:
hrr_counties_joined_gpd = gpd.sjoin(county_pop_gdf, hrr_gdf, how='left', op='intersects')


In [23]:
def calculate_ratio(row):
    if row['hrr_geom'] is None:
        return 0.0
    i = row['hrr_geom'].buffer(0).intersection(row['geometry'].buffer(0))
    return i.area / row['geometry'].area

hrr_counties_joined_gpd['ratio'] = hrr_counties_joined_gpd.apply(calculate_ratio, axis=1)


In [24]:
for column in county_pops.keys():    
    hrr_counties_joined_gpd[column] = \
        (hrr_counties_joined_gpd[column] * hrr_counties_joined_gpd['ratio']).round()


In [25]:
hrr_pops = hrr_counties_joined_gpd.groupby('HRR_BDRY_I')[list(county_pops.keys())].sum()
hrr_pops

,Population,Population (20+),Population (65+)
HRR_BDRY_I,,,
1.0,2315317.0,1739960.0,394851.0
2.0,369295.0,280167.0,69067.0
3.0,664860.0,500270.0,106495.0
4.0,856229.0,640362.0,148917.0
5.0,459289.0,342900.0,73128.0
...,...,...,...
302.0,419904.0,318991.0,82710.0
303.0,2715441.0,2018636.0,425747.0
304.0,196589.0,150628.0,35596.0


In [26]:
enriched_hrr_gdf = hrr_gdf.join(hrr_pops, on='HRR_BDRY_I').fillna(value=0)
enriched_hrr_gdf = enriched_hrr_gdf.drop('hrr_geom', axis=1).to_crs('EPSG:4326')

In [27]:
enriched_hrr_gdf

,HRR_BDRY_I,HRRNUM,HRRCITY,geometry,Population,Population (20+),Population (65+)
0,1,1,AL- BIRMINGHAM,"MULTIPOLYGON (((-85.89658 32.70483, -85.90771 ...",2315317.0,1739960.0,394851.0
1,2,2,AL- DOTHAN,"MULTIPOLYGON (((-86.19400 31.43991, -86.19478 ...",369295.0,280167.0,69067.0
2,3,5,AL- HUNTSVILLE,"MULTIPOLYGON (((-86.69474 34.42309, -86.71003 ...",664860.0,500270.0,106495.0
3,4,6,AL- MOBILE,"MULTIPOLYGON (((-87.55554 30.47282, -87.56131 ...",856229.0,640362.0,148917.0
4,5,7,AL- MONTGOMERY,"POLYGON ((-86.49925 31.52517, -86.64228 31.523...",459289.0,342900.0,73128.0
...,...,...,...,...,...,...,...
301,302,450,WI- MARSHFIELD,"POLYGON ((-91.00410 44.70601, -91.00881 44.708...",419904.0,318991.0,82710.0
302,303,451,WI- MILWAUKEE,"POLYGON ((-87.80049 42.49190, -87.79799 42.471...",2715441.0,2018636.0,425747.0
303,304,452,WI- NEENAH,"MULTIPOLYGON (((-88.88595 43.63361, -88.92585 ...",196589.0,150628.0,35596.0
304,305,456,WI- WAUSAU,"POLYGON ((-89.30141 45.46939, -89.30115 45.454...",160443.0,124978.0,34252.0


In [28]:
enriched_hrr_gdf.to_file(processed_data_path('us_hrr_with_pop.geojson'), driver='GeoJSON')